### Pedro Gonçalves (A82313) & Roberto Cachada (A81012)

Os algoritmos implementados neste *notebook* são baseados no [documento](https://ntruprime.cr.yp.to/nist/ntruprime-20190330.pdf) da segunda ronda da candidatura ao concurso da NIST.

##### PKE-IND-CCA

A camada do **Streamlined NTRU Prime** que implementa um PKE determinista é a camada interna, denominada **Streamlined NTRU Prime Core**.

In [23]:
#from sage.all_cmdline import *
import sys
import hashlib
def sha512(s):
    h = hashlib.sha512()
    h.update(s)
    return h.digest()

_sage_const_0 = Integer(0); _sage_const_2 = Integer(2); _sage_const_1 = Integer(1); _sage_const_761 = Integer(761); _sage_const_4591 = Integer(4591); _sage_const_286 = Integer(286); _sage_const_250 = Integer(250); _sage_const_292 = Integer(292); _sage_const_2156 = Integer(2156); _sage_const_114 = Integer(114); _sage_const_2007 = Integer(2007); _sage_const_287 = Integer(287); _sage_const_3 = Integer(3); _sage_const_4 = Integer(4); _sage_const_653 = Integer(653); _sage_const_4621 = Integer(4621); _sage_const_288 = Integer(288); _sage_const_5 = Integer(5); _sage_const_252 = Integer(252); _sage_const_289 = Integer(289); _sage_const_2175 = Integer(2175); _sage_const_113 = Integer(113); _sage_const_2031 = Integer(2031); _sage_const_290 = Integer(290); _sage_const_6 = Integer(6); _sage_const_857 = Integer(857); _sage_const_5167 = Integer(5167); _sage_const_322 = Integer(322); _sage_const_7 = Integer(7); _sage_const_281 = Integer(281); _sage_const_329 = Integer(329); _sage_const_2433 = Integer(2433); _sage_const_101 = Integer(101); _sage_const_2265 = Integer(2265); _sage_const_324 = Integer(324); _sage_const_16 = Integer(16); _sage_const_256 = Integer(256); _sage_const_8 = Integer(8); _sage_const_14 = Integer(14); _sage_const_15 = Integer(15); _sage_const_32 = Integer(32); _sage_const_65536 = Integer(65536); _sage_const_16777216 = Integer(16777216); _sage_const_0x3fffffff = Integer(0x3fffffff); _sage_const_30 = Integer(30); _sage_const_128 = Integer(128); _sage_const_64 = Integer(64); _sage_const_16384 = Integer(16384); _sage_const_255 = Integer(255); _sage_const_6144 = Integer(6144); _sage_const_1218 = Integer(1218); _sage_const_1536 = Integer(1536); _sage_const_1015 = Integer(1015); _sage_const_10 = Integer(10); _sage_const_100 = Integer(100)

#Parámetros
(round1,p,q,w,lpr) = (True,_sage_const_761 ,_sage_const_4591 ,_sage_const_286 ,False)
assert p.is_prime()
assert q.is_prime()
assert w > Integer(0)
assert Integer(2)*p >= Integer(3)*w
assert q >= Integer(16)*w+Integer(1)

#-------

F3 = GF(_sage_const_3 )
def ZZ_fromF3(c):
    assert c in F3
    return ZZ(c+Integer(1))-Integer(1)

Fq = GF(q)
q12 = ZZ((q-_sage_const_1 )/_sage_const_2 )
def ZZ_fromFq(c):
    assert c in Fq
    return ZZ(c+q12)-q12
#-------

Zx = ZZ['x']; (x,) = Zx._first_ngens(1)
R = Zx.quotient(x**p-x-Integer(1) , names=('xp',)); (xp,) = R._first_ngens(1)

def Weightw_is(r):
    assert r in R
    return w == len([i for i in range(p) if r[i] != _sage_const_0 ])

def Small_is(r):
    assert r in R
    return all(abs(r[i]) <= _sage_const_1  for i in range(p))

def Short_is(r):
    return Small_is(r) and Weightw_is(r)

#------- polynomials mod 3

F3x = F3['x3']; (x3,) = F3x._first_ngens(1)
R3 = F3x.quotient(x**p-x-_sage_const_1 , names=('x3p',)); (x3p,) = R3._first_ngens(1)

def R_fromR3(r):
    assert r in R3
    return R([ZZ_fromF3(r[i]) for i in range(p)])

def R3_fromR(r):
    assert r in R
    return R3([r[i] for i in range(p)])
    
# ----- polynomials mod q

Fqx = Fq['xq']; (xq,) = Fqx._first_ngens(1)
Rq = Fqx.quotient(x**p-x-_sage_const_1 , names=('xqp',)); (xqp,) = Rq._first_ngens(1)
assert (xq**p-xq-_sage_const_1 ).is_irreducible()

def R_fromRq(r):
    assert r in Rq
    return R([ZZ_fromFq(r[i]) for i in range(p)])

def Rq_fromR(r):
    assert r in R
    return Rq([r[i] for i in range(p)])

# ----- rounded polynomials mod q

def Rounded_is(r):
    assert r in R
    return (all(r[i]%_sage_const_3  == _sage_const_0  for i in range(p))
        and all(r[i] >= -q12 for i in range(p))
        and all(r[i] <= q12 for i in range(p)))

def Round(a):
    assert a in Rq
    c = R_fromRq(a)
    r = [_sage_const_3 *round(c[i]/_sage_const_3 ) for i in range(p)]
    assert all(abs(r[i]-c[i]) <= _sage_const_1  for i in range(p))
    r = R(r)
    assert Rounded_is(r)
    return r

# ----- sorting to generate short polynomial

def Short_fromlist(L): # L is list of p uint32
    L = [L[i]&-_sage_const_2  for i in range(w)] + [(L[i]&-_sage_const_3 )|_sage_const_1  for i in range(w,p)]
    assert all(L[i]%_sage_const_2  == _sage_const_0  for i in range(w))
    assert all(L[i]%_sage_const_4  == _sage_const_1  for i in range(w,p))
    L.sort()
    L = [(L[i]%_sage_const_4 )-_sage_const_1  for i in range(p)]
    assert all(abs(L[i]) <= _sage_const_1  for i in range(p))
    assert sum(abs(L[i]) for i in range(p)) == w
    r = R(L)
    assert Short_is(r)
    return r

def random8(): return randrange(_sage_const_256 )

def urandom32():
    c0 = random8()
    c1 = random8()
    c2 = random8()
    c3 = random8()
    return c0 + _sage_const_256 *c1 + _sage_const_65536 *c2 + _sage_const_16777216 *c3

def Short_random(): # R element with w coeffs +-1
    L = [urandom32() for i in range(p)]
    return Short_fromlist(L)

def randomrange3():
    return ((urandom32() & _sage_const_0x3fffffff ) * _sage_const_3 ) >> _sage_const_30 

def Small_random():
    r = R([randomrange3()-_sage_const_1  for i in range(p)])
    assert Small_is(r)
    return r

In [24]:
def KeyGen():
    while (True):
        g = Small_random()
        if R3_fromR(g).is_unit(): break
    f = Short_random()
    h = Rq_fromR(g)/Rq_fromR(_sage_const_3 *f)
    return h,(f,_sage_const_1 /R3_fromR(g))

def Encrypt(r,h):
    assert Short_is(r)
    assert h in Rq
    return Round(h*Rq_fromR(r))

def Decrypt(c,k):
    f,v = k
    assert Rounded_is(c)
    assert Short_is(f)
    assert v in R3
    e = R3_fromR(R_fromRq(_sage_const_3 *Rq_fromR(f)*Rq_fromR(c)))
    r = R_fromR3(e*v)
    if Weightw_is(r): return r
    return R([_sage_const_1 ]*w+[_sage_const_0 ]*(p-w))

In [34]:
h,j = KeyGen()
r = Short_random()
ct = Encrypt(r,h)
rr = Decrypt(ct,j)
print(r==rr)

True


##### KEM-IND-CPA

Já a camada do **Streamlined NTRU Prime** que implementa um KEM é a camada externa, denominada **Streamlined NTRU Prime**.